In [29]:
import cudf
import pandas as pd
import numpy as np
import cupy as cp
from catboost import CatBoostClassifier
import pickle
import sys
import gc

# Parameters

In [30]:
#input_dir = '../../04_LearningPrediction/inputs/'
input_dir = 'inputs/'
ind_list = [1,2, 3, 4, 5, 6, 7, 8, 9, 10, 11,12, 13, 14, 15, 16, 17, 18]
n_fold = 10
fold = 0
kol_col_start = 16

# Load Train Data and Labels

In [31]:
dtypes = {"session_id": 'int64',
          "index": np.int16,
          "elapsed_time": np.int32,
          "event_name": 'category',
          "name": 'category',
          "level": np.int8,
          "page": np.float32,
          "room_coor_x": np.float32,
          "room_coor_y": np.float32,
          "screen_coor_x": np.float32,
          "screen_coor_y": np.float32,
          "hover_duration": np.float32,
          "text": 'category',
          "fqid": 'category',
          "room_fqid": 'category',
          "text_fqid": 'category',
          "fullscreen": np.int8,
          "hq": np.int8,
          "music": np.int8,
          "level_group": 'category'
          }

use_col = ['session_id', 'index', 'elapsed_time', 'event_name', 'name', 'level', 'page',
           'room_coor_x', 'room_coor_y', 'hover_duration', 'text', 'fqid', 'room_fqid', 'text_fqid', 'level_group']

In [32]:
targets = pd.read_csv(input_dir + 'predict-student-performance-from-game-play/train_labels.csv')
targets['session'] = targets.session_id.apply(lambda x: int(x.split('_')[0]) )
targets['q'] = targets.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )
print( targets.shape )
targets.head()

(424116, 4)


,session_id,correct,session,q
0,20090312431273200_q1,1,20090312431273200,1
1,20090312433251036_q1,0,20090312433251036,1
2,20090312455206810_q1,1,20090312455206810,1
3,20090313091715820_q1,0,20090313091715820,1
4,20090313571836404_q1,1,20090313571836404,1


In [33]:
feature_df = pd.read_csv(input_dir + 'featur/feature_sort.csv')

# Feature Engineer

In [34]:
def delt_time_def(df):
    df = df.sort_values(by=['session_id', 'elapsed_time'])
    df['d_time'] = df['elapsed_time'].diff(1)
    df['d_time'].fillna(0, inplace=True)
    df['delt_time'] = df['d_time'].clip(0, 103000)
    df['delt_time_next'] = df['delt_time'].shift(-1)
    #df['d_room_coor_x'] = df['room_coor_x'].diff(1)
    #df['d_room_coor_y'] = df['room_coor_y'].diff(2)
    #df['dd_time'] = df['d_time'].diff(1)
    df['d_hover_duration'] = df['hover_duration'].diff(1)
    df['d_hover_duration'].fillna(0, inplace=True)
    return df

In [35]:
def feature_engineer(train, kol_f):
    global kol_col, kol_col_max
    kol_col = kol_col_start
    kol_col_max = kol_col_start+2+kol_f*2
    col = [i for i in range(0,kol_col_max)]
    new_train = cudf.DataFrame(index=train['session_id'].unique(), columns=col, dtype=np.float32)  
    new_train[kol_col+1] = new_train.index # "session_id"    
    new_train_last = new_train[kol_col+1].to_pandas()
    new_train[0] = train.groupby(['session_id'])['d_time'].quantile(q=0.3)
    new_train[1] = train.groupby(['session_id'])['d_time'].quantile(q=0.5)
    new_train[2] = train.groupby(['session_id'])['d_time'].quantile(q=0.65)
    new_train[3] = train.groupby(['session_id'])['d_time'].quantile(q=0.8)
    new_train[4] = train.groupby(['session_id'])['hover_duration'].agg('mean')
    new_train[5] = train.groupby(['session_id'])['hover_duration'].agg('std')    
    
    new_train[6] = train.groupby(['session_id'])['room_coor_x'].agg('mean')
    new_train[7] = train.groupby(['session_id'])['room_coor_x'].agg('std')
    new_train[8] = train.groupby(['session_id'])['room_coor_y'].agg('mean')
    new_train[9] = train.groupby(['session_id'])['room_coor_y'].agg('std')    
    new_train[10] = train.groupby(['session_id'])['level'].agg('first')
    new_train[11] = train.groupby(['session_id'])['level'].agg('last')      
    new_train[12] = train.groupby(['session_id'])['fqid'].agg('nunique') 

 

    new_train[kol_col-3] = new_train_last.apply(lambda x: int(str(x)[:2])).astype(np.uint8) # "year"
    new_train[kol_col-2] = new_train_last.apply(lambda x: int(str(x)[2:4])+1).astype(np.uint8) # "month"
    new_train[kol_col-1] = new_train_last.apply(lambda x: int(str(x)[4:6])).astype(np.uint8) # "day"
    new_train[kol_col+0] = new_train_last.apply(lambda x: int(str(x)[6:8])).astype(np.uint8) + new_train_last.apply(lambda x: int(str(x)[8:10])).astype(np.uint8)/60
    new_train[kol_col+1] = 0
    new_train = new_train.fillna(-1)
    
    return new_train

In [36]:
def feature_next_t(row_f, new_train, train, gran_1, gran_2, i):
    global kol_col
    kol_col +=1
    col1 = row_f['col1']
    val1 = row_f['val1']

    try:
        maska = (train[col1] == val1)
    except:
        maska = (train[col1].to_pandas() ==val1)

    if row_f['kol_col'] == 1:       
        new_train[kol_col] = train[maska].groupby(['session_id'])['delt_time_next'].sum()
        if gran_1:
            kol_col +=1
            new_train[kol_col] = train[maska].groupby(['session_id'])['delt_time'].mean()
        if gran_2:
            kol_col +=1
            new_train[kol_col] = train[maska].groupby(['session_id'])['index'].count()                            
    elif row_f['kol_col'] == 2: 
        col2 = row_f['col2']
        val2 = row_f['val2']

        # to deal with exception due to the nan
        try:
            maskb = train[col2]==val2
        except:
            maskb = maska.copy()
            maskb[:] = False

        try:
            maska = maska & maskb    
        except:
            maska[:] = False
            
        new_train[kol_col] = train[maska].groupby(['session_id'])['delt_time_next'].sum()
        if gran_1:
            kol_col +=1
            new_train[kol_col] = train[maska].groupby(['session_id'])['delt_time'].mean()
        if gran_2:
            kol_col +=1
            new_train[kol_col] = train[maska].groupby(['session_id'])['index'].count()
      
    return new_train


In [37]:
def feature_engineer_new(new_train, train, feature_q, kol_f):
    g1 = 0.7
    g2 = 0.3 
    gran1 = round(kol_f * g1)
    gran2 = round(kol_f * g2)    

    for i in range(0, kol_f): 
        row_f = feature_q.loc[i]  
        new_train = feature_next_t(row_f, new_train, train, i < gran1, i < gran2, i)         
    return new_train



In [38]:
def feature_quest(new_train, train, quest, kol_f):
    global kol_col
    kol_col = kol_col_start
    feature_q = feature_df[feature_df['quest'] == quest].copy()

    feature_q.reset_index(drop=True, inplace=True)
    new_train = feature_engineer_new(new_train, train, feature_q, kol_f)
    col = [i for i in range(0,kol_col+1)]
    return new_train[col]


In [39]:
def create_dataset(old_train, quests, models, list_kol_f):
    
    kol_quest = len(quests)
    # ITERATE THRU QUESTIONS
    for q in quests:
        print('### quest ', q, end='')
        new_train = feature_engineer(old_train, list_kol_f[q])
        train_x = feature_quest(new_train, old_train, q, list_kol_f[q])
        print (' ---- ', 'train_q.shape = ', train_x.shape)
            
        # TRAIN DATA
        train_users = train_x.index.values
        train_y = targets.loc[targets.q==q].set_index('session').loc[train_users.get()]
        train_y = np.array(train_y['correct'].astype('float32'))
        train_y = train_y.reshape(-1, 1)

        # Save Data
        train_x = train_x.to_pandas()
        data = np.concatenate([train_x.astype('float32'), train_y], axis = 1)
        with open(input_dir + f'preprocessed/{q}.npy', 'wb') as f:
            np.save(f, data)
    
    return models

In [40]:
models = {}
best_threshold = 0.63

In [41]:
list_kol_f = {
    1:150, 2:100, 3:110, 
    4:100, 5:190, 6:150, 7:180, 8:110, 9:180, 10:160, 11:180, 12:140, 13:150,
    14: 230, 15:160, 16:130, 17:230, 18:160   
             }

for i in range(1, 19):
    list_kol_f[i] = 230

In [42]:
df0_4 = cudf.read_csv(input_dir + 'featur/train_0_4t.csv', dtype=dtypes) 
kol_lvl = (df0_4 .groupby(['session_id'])['level'].agg('nunique') < 5)
list_session = kol_lvl[kol_lvl].index
df0_4  = df0_4 [~df0_4 ['session_id'].isin(list_session)]
df0_4 = delt_time_def(df0_4)

quests_0_4 = [1, 2, 3] 
#list_kol_f = {1:140,3:110}
models = create_dataset(df0_4, quests_0_4, models, list_kol_f)
del df0_4


### quest  1

 ----  train_q.shape =  (23562, 477)
### quest  2

In [ ]:
df5_12 = cudf.read_csv(input_dir + 'featur/train_5_12t.csv', dtype=dtypes)
kol_lvl = (df5_12.groupby(['session_id'])['level'].agg('nunique') < 8)
list_session = kol_lvl[kol_lvl].index
df5_12 = df5_12[~df5_12['session_id'].isin(list_session)]
df5_12 = delt_time_def(df5_12)
quests_5_12 = [4, 5, 6, 7, 8, 9, 10, 11, 12] 
# list_kol_f = {4:110, 5:220, 6:120, 7:110, 8:110, 9:100, 10:140, 11:120}

models = create_dataset(df5_12, quests_5_12, models, list_kol_f)
del df5_12
gc.collect()

### quest  4 ----  train_q.shape =  (23561, 477)
### quest  5 ----  train_q.shape =  (23561, 477)
### quest  6 ----  train_q.shape =  (23561, 477)
### quest  7 ----  train_q.shape =  (23561, 477)
### quest  8 ----  train_q.shape =  (23561, 477)
### quest  9 ----  train_q.shape =  (23561, 477)
### quest  10 ----  train_q.shape =  (23561, 477)
### quest  11 ----  train_q.shape =  (23561, 477)
### quest  12 ----  train_q.shape =  (23561, 477)
***


0

In [ ]:
df13_22 = cudf.read_csv(input_dir + 'featur/train_13_22t.csv', dtype=dtypes) 
kol_lvl = (df13_22 .groupby(['session_id'])['level'].agg('nunique') < 10)
list_session = kol_lvl[kol_lvl].index
df13_22  = df13_22 [~df13_22 ['session_id'].isin(list_session)]
df13_22 = delt_time_def(df13_22)

quests_13_22 = [13, 14, 15, 16, 17, 18] 
# list_kol_f = {14: 160, 15:160, 16:105, 17:140}

models = create_dataset(df13_22, quests_13_22, models, list_kol_f)
del df13_22
gc.collect()


### quest  13 ----  train_q.shape =  (22986, 477)
### quest  14 ----  train_q.shape =  (22986, 477)
### quest  15 ----  train_q.shape =  (22986, 477)
### quest  16 ----  train_q.shape =  (22986, 477)
### quest  17 ----  train_q.shape =  (22986, 477)
### quest  18 ----  train_q.shape =  (22986, 477)
***


0

# Score and Metric

In [ ]:
def calc_f1_score(pred, target, threshold):
    th_pred = pred >= threshold
    precision = np.sum(target*th_pred) / (np.sum(th_pred)+1e-6)
    recall    = np.sum(target*th_pred) / np.sum(target)

    if (precision==0.0):
        return 0
    t_f1_score = 2/(1/precision + 1/recall)
    
    n_target = 1-target
    #th_pred = pred >= threshold
    th_pred = 1-th_pred
    precision = np.sum(n_target*th_pred) / (np.sum(th_pred)+1e-6)
    recall    = np.sum(n_target*th_pred) / np.sum(n_target)
    if (precision==0.0):
        return 0
    f_f1_score = 2/(1/precision + 1/recall)
    
    macro_f1_score = (t_f1_score+f_f1_score)/2
    
    return macro_f1_score

def get_best_f1_threshold(pred, target, thres_inter = 0.01):
    threshold = np.arange(0, 1.00, thres_inter)
    score_list = np.zeros(len(threshold))
    for i in range(0, len(threshold)):
        score_list[i] = calc_f1_score(pred, target, threshold[i])
    best_ind = np.argmax(score_list)
    best_threshold = best_ind*thres_inter
    best_score = score_list[best_ind]
    
    return best_score, best_threshold

class CatBoostEvalMetricMacroF1(object):
    def get_best_f1_threshold(self, pred, target):
        threshold = np.arange(0, 1.00, 0.01)
        score_list = np.zeros(len(threshold))
        for i in range(0, len(threshold)):
            score_list[i] = calc_f1_score(pred, target, threshold[i])
        best_ind = np.argmax(score_list)
        best_threshold = best_ind*0.01
        best_score = score_list[best_ind]
        return best_score, best_threshold
    
    def get_final_error(self, error, weight):
        return error

    def is_max_optimal(self):
        # the larger metric value the better
        return True

    def evaluate(self, approxes, target, weight):
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])
        preds = np.array(approxes[0])
        target = np.array(target)
        best_score, best_thres = self.get_best_f1_threshold(preds, target)
        return best_score, 0

# Train

In [ ]:
def load_npy(ind, n_fold = n_fold):
    with open(input_dir + f'preprocessed/{ind}.npy', 'rb') as f:
        xy = np.load(f)    
    xy[np.isnan(xy)] = -1
    
    with open(input_dir + f'preprocessed/{n_fold}fold/{ind}_fold.npy', 'rb') as f:
        fold = np.load(f).astype(int)
    return xy, fold

In [ ]:
def train_catboost(ind, fold, n_fold = n_fold, exclude_fold = None, learning_rate = 0.02,
                  depth = 6, l2_leaf_reg = 0.1, bagging_temperature = 0.5):
    xy, fold_arr = load_npy(ind, n_fold)
    

    model = CatBoostClassifier(
    n_estimators = 1000000,
    learning_rate= learning_rate,
    depth = depth,
    l2_leaf_reg = l2_leaf_reg,
    bagging_temperature = bagging_temperature,
    task_type = 'CPU',
    eval_metric = CatBoostEvalMetricMacroF1(),
    )
    

    model.fit(xy[(fold_arr!=fold),:-1], xy[fold_arr!=fold,-1], 
          eval_set = (xy[fold_arr==fold,:-1], xy[fold_arr==fold,-1]), 
          verbose = 100000, 
          early_stopping_rounds = 500, 
          use_best_model = True,
            )

    pred = model.predict_proba(xy[fold_arr==fold,:-1])[:,1]
    best_score, best_threshold = get_best_f1_threshold(pred, xy[fold_arr==fold, -1])
        
    print(f'ind{ind}_best_score: {best_score}')
    print(f'ind{ind}_best_thres: {best_threshold}')
    model.save_model(f'weights/cat_model_{ind}_f{fold}.bin')
    
    target = xy[fold_arr==fold,-1].copy()
    del model, xy
    gc.collect()
    
    return best_threshold, best_score, pred, target

In [ ]:
preds   = []
targets = []

for ind in ind_list:
    thres, score, pred, target = train_catboost(ind, 0)
    preds.append(pred)
    targets.append(target)

preds   = np.concatenate(preds, axis = 0)
targets = np.concatenate(targets, axis = 0)


0:	learn: 0.6859929	test: 0.6859242	best: 0.6859242 (0)	total: 82.3ms	remaining: 49.3s
599:	learn: 0.4376448	test: 0.4887725	best: 0.4885098 (564)	total: 15.2s	remaining: 0us

bestTest = 0.4885098443
bestIteration = 564

Shrink model to first 565 iterations.
ind1_best_score: 0.6890240084717365
ind1_best_thres: 0.65
0:	learn: 0.6556076	test: 0.6556574	best: 0.6556574 (0)	total: 74.3ms	remaining: 44.5s
599:	learn: 0.0505065	test: 0.0959392	best: 0.0947591 (442)	total: 16.3s	remaining: 0us

bestTest = 0.09475914216
bestIteration = 442

Shrink model to first 443 iterations.
ind2_best_score: 0.6384708380408124
ind2_best_thres: 0.9
0:	learn: 0.6705127	test: 0.6707089	best: 0.6707089 (0)	total: 50.4ms	remaining: 30.2s
599:	learn: 0.1685516	test: 0.2244642	best: 0.2233620 (518)	total: 10.6s	remaining: 0us

bestTest = 0.2233619798
bestIteration = 518

Shrink model to first 519 iterations.
ind3_best_score: 0.6132164364277979
ind3_best_thres: 0.88
0:	learn: 0.6816188	test: 0.6821729	best: 0.68217

In [ ]:
best_score, best_threshold = get_best_f1_threshold(preds, targets, 0.001)
print(f'Best score:     {best_score}')
print(f'Best threshold: {best_threshold}')

Best score:     0.6924199840845655
Best threshold: 0.63
